In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import seaborn as sns
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
'''
Tried these features not much difference: play around with them if you want. 



df['log_popularity'] = np.log1p(df['popularity'])
test['log_popularity'] = np.log1p(test['popularity'])
df['log_budget'] = np.log1p(df['budget'])

df['ratio_budget_runtime'] = (df['log_budget'] / df['runtime'])
df['ratio_budget_popularity'] = df['log_budget'] / df['log_popularity']
df['ratio_budget_year'] = df['log_budget'] / df['release_year'] #
df['budget_to_mean_year'] = df['log_budget'] / df.groupby('release_year')['log_budget'].transform('mean')
df['ratio_popularity_year'] = df['log_popularity'] / df['release_year']
df['popularity_to_mean_year'] = df['log_popularity'] / df.groupby('release_year')['log_popularity'].transform('mean')

df['ratio_budget_year'].fillna(df['ratio_budget_year'].mean(),inplace=True)
df['budget_to_mean_year'].fillna(df['budget_to_mean_year'].mean(),inplace=True) #fill na with test vals?
df['ratio_budget_runtime'].fillna(df['ratio_budget_runtime'].mean(),inplace=True) #fill na with test vals?

le = LabelEncoder()
df['prodcomp']=le.fit_transform(df['production_companies'])
test['prodcomp']=le.fit_transform(test['production_companies'])

gender of cast feature might help
'''

"\nTried these features not much difference: play around with them if you want. \n\n\n\ndf['log_popularity'] = np.log1p(df['popularity'])\ntest['log_popularity'] = np.log1p(test['popularity'])\ndf['log_budget'] = np.log1p(df['budget'])\n\ndf['ratio_budget_runtime'] = (df['log_budget'] / df['runtime'])\ndf['ratio_budget_popularity'] = df['log_budget'] / df['log_popularity']\ndf['ratio_budget_year'] = df['log_budget'] / df['release_year'] #\ndf['budget_to_mean_year'] = df['log_budget'] / df.groupby('release_year')['log_budget'].transform('mean')\ndf['ratio_popularity_year'] = df['log_popularity'] / df['release_year']\ndf['popularity_to_mean_year'] = df['log_popularity'] / df.groupby('release_year')['log_popularity'].transform('mean')\n\ndf['ratio_budget_year'].fillna(df['ratio_budget_year'].mean(),inplace=True)\ndf['budget_to_mean_year'].fillna(df['budget_to_mean_year'].mean(),inplace=True) #fill na with test vals?\ndf['ratio_budget_runtime'].fillna(df['ratio_budget_runtime'].mean(),inpl

In [3]:
df = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2train.csv", header=0) 
df.head()
test = pd.read_csv(r"C:\Users\Ben\Desktop\moviedata\proj2test.csv")

In [4]:
test.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew
0,3001,"[{'id': 34055, 'name': 'Pokémon Collection', '...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",http://www.pokemon.com/us/movies/movie-pokemon...,tt1226251,ja,ディアルガVSパルキアVSダークライ,Ash and friends (this time accompanied by newc...,3.851534,...,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",7/14/07,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Somewhere Between Time & Space... A Legend Is ...,Pokémon: The Rise of Darkrai,"[{'id': 11451, 'name': 'pok√©mon'}, {'id': 115...","[{'cast_id': 3, 'character': 'Tonio', 'credit_...","[{'credit_id': '52fe44e7c3a368484e03d683', 'de..."
1,3002,NaN,88000,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,tt0051380,en,Attack of the 50 Foot Woman,When an abused wife grows to giant size becaus...,3.559789,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/19/58,65.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A titanic beauty spreads a macabre wave of hor...,Attack of the 50 Foot Woman,"[{'id': 9748, 'name': 'revenge'}, {'id': 9951,...","[{'cast_id': 2, 'character': 'Nancy Fowler Arc...","[{'credit_id': '55807805c3a3685b1300060b', 'de..."
2,3003,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,tt0118556,en,Addicted to Love,Good-natured astronomer Sam is devastated when...,8.085194,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/23/97,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Comedy About Lost Loves And Last Laughs,Addicted to Love,"[{'id': 931, 'name': 'jealousy'}, {'id': 9673,...","[{'cast_id': 11, 'character': 'Maggie', 'credi...","[{'credit_id': '52fe4330c3a36847f8041367', 'de..."
3,3004,NaN,6800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",http://www.sonyclassics.com/incendies/,tt1255953,fr,Incendies,A mother's last wishes send twins Jeanne and S...,8.596012,...,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",9/4/10,130.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,The search began at the opening of their mothe...,Incendies,"[{'id': 378, 'name': 'prison'}, {'id': 539, 'n...","[{'cast_id': 6, 'character': 'Nawal', 'credit_...","[{'credit_id': '56478092c3a36826140043af', 'de..."
4,3005,NaN,2000000,"[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",NaN,tt0418753,en,Inside Deep Throat,"In 1972, a seemingly typical shoestring budget...",3.217680,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2/11/05,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It was filmed in 6 days for 25 thousand dollar...,Inside Deep Throat,"[{'id': 279, 'name': 'usa'}, {'id': 1228, 'nam...","[{'cast_id': 1, 'character': 'Narrator (voice)...","[{'credit_id': '52fe44ce9251416c75041967', 'de..."


In [5]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [6]:
df['revenue'] = np.log1p(df['revenue'])

In [7]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [8]:
missing_values_table(df)
# we have a good amount of missing variables that have to be either dropped or filled as it is a kaggle competition and we cannot 
# drop rows on the test set

Your selected dataframe has 24 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
belongs_to_collection,2396,79.9
homepage,2054,68.5
tagline,597,19.9
Keywords,276,9.2
production_companies,156,5.2
production_countries,55,1.8
spoken_languages,20,0.7
crew,16,0.5
cast,13,0.4
overview,8,0.3


In [9]:
df['isbelongs_to_collectionNA'] = 0
df.loc[df['belongs_to_collection'].isnull(), 'isbelongs_to_collectionNA'] = 1

test['isbelongs_to_collectionNA'] = 0
test.loc[test['belongs_to_collection'].isnull(), 'isbelongs_to_collectionNA'] = 1

In [10]:
df['isTaglineNA'] = 0
df.loc[df['tagline'].isnull(), 'isTaglineNA'] = 1
test['isTaglineNA'] = 0
test.loc[test['tagline'].isnull(), 'isTaglineNA'] = 1

In [11]:
#not used
df = df.drop(["homepage","belongs_to_collection", "poster_path", 'tagline','Keywords'], axis = 1)
test = test.drop(["homepage","belongs_to_collection", "poster_path",'tagline','Keywords'], axis = 1)
test['status'].fillna('Released',inplace=True)

In [12]:
le = LabelEncoder()
df['status']=le.fit_transform(df['status'])
test['status']=le.fit_transform(test['status'])


In [13]:
test['status']=le.fit_transform(test['status'])

In [14]:
df['original_language']=le.fit_transform(df['original_language'])
test['original_language']=le.fit_transform(test['original_language'])

In [15]:
df['cast'].fillna('0',inplace=True)
df['crew'].fillna('0',inplace=True)

test['cast'].fillna('0',inplace=True)
test['crew'].fillna('0',inplace=True)
# since we count crew and cast alter it is fine to just fill with as we it wont affect the feature

In [16]:
print (df['genres'].mode()[0])
print (df['spoken_languages'].mode()[0])
print (df['production_companies'].mode()[0])
print (df['production_countries'].mode()[0])

[{'id': 18, 'name': 'Drama'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'name': 'Paramount Pictures', 'id': 4}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]


In [17]:
df['genres'].fillna(df['genres'].mode()[0], inplace=True)
df['spoken_languages'].fillna(df['spoken_languages'].mode()[0], inplace=True)
df['production_companies'].fillna(df['production_companies'].mode()[0], inplace=True)
df['production_countries'].fillna(df['production_countries'].mode()[0], inplace=True)
#test
test['genres'].fillna(test['genres'].mode()[0], inplace=True)
test['spoken_languages'].fillna(test['spoken_languages'].mode()[0], inplace=True)
test['production_companies'].fillna(test['production_companies'].mode()[0], inplace=True)
test['production_countries'].fillna(test['production_countries'].mode()[0], inplace=True)
#print (df['spoken_languages'].mode()[0])
#print (df['production_companies'].mode()[0])
#print (df['production_countries'].mode()[0])

In [18]:
missing_values_table(df)

Your selected dataframe has 21 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
overview,8,0.3
runtime,2,0.1


In [19]:
df["overview"].fillna(0)
test["overview"].fillna(0)

0       Ash and friends (this time accompanied by newc...
1       When an abused wife grows to giant size becaus...
2       Good-natured astronomer Sam is devastated when...
3       A mother's last wishes send twins Jeanne and S...
4       In 1972, a seemingly typical shoestring budget...
                              ...                        
4393    A young man is plunged into a life of subterfu...
4394    Three friends, whose lives have been drifting ...
4395    Frank Galvin is a down-on-his luck lawyer, red...
4396    For 19-year-old Jay, fall should be about scho...
4397    Twelve episodic tales in the life of a Parisia...
Name: overview, Length: 4398, dtype: object

In [20]:
test['runtime'].fillna(test['runtime'].mean(),inplace=True)
df['runtime'].fillna(test['runtime'].mean(),inplace=True)

In [22]:
print(test[test['release_date'].isnull()])
print(test[test['year'].isnull()])


       id  budget                         genres    imdb_id  \
828  3829       0  [{'id': 18, 'name': 'Drama'}]  tt0210130   

     original_language              original_title  \
828                  9  Jails, Hospitals & Hip-Hop   

                                              overview  popularity  \
828  Jails, Hospitals &amp; Hip-Hop is a cinematic ...    0.009057   

                          production_companies  \
828  [{'name': 'Paramount Pictures', 'id': 4}]   

                                  production_countries release_date  runtime  \
828  [{'iso_3166_1': 'US', 'name': 'United States o...          NaN     90.0   

                             spoken_languages  status  \
828  [{'iso_639_1': 'en', 'name': 'English'}]       1   

                          title cast crew  year  isbelongs_to_collectionNA  \
828  Jails, Hospitals & Hip-Hop   []    0   NaN                          1   

     isTaglineNA  
828            0  
       id  budget                         genres   

In [23]:
test['release_date'].fillna('3/20/01',inplace=True)

In [24]:
test['year'].fillna("2000", inplace = True)

In [25]:
test['title'].fillna(test['original_title'],inplace=True)

In [26]:
missing_values_table(test)

Your selected dataframe has 20 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
overview,14,0.3


In [27]:
df['year'] = pd.DatetimeIndex(df['release_date']).year
test['year'] = pd.DatetimeIndex(test['release_date']).year

In [28]:
Y = df["revenue"]

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   overview                   2992 non-null   object 
 7   popularity                 3000 non-null   float64
 8   production_companies       3000 non-null   object 
 9   production_countries       3000 non-null   object 
 10  release_date               3000 non-null   object 
 11  runtime                    3000 non-null   float64
 12  spoken_languages           3000 non-null   object 
 13  status                     3000 non-null   int32

In [30]:
Y.head()

0    16.326300
1    18.370959
2    16.387512
3    16.588099
4    15.182615
Name: revenue, dtype: float64

In [31]:
#df["IsCollection"].value_counts()

In [33]:
df["budget_year_ratio"] = df['budget'] / (df['year'] * df['year'])
test["budget_year_ratio"] = test['budget'] / (test['year'] * test['year'])

In [35]:
#little function for for finding an overview
def extractOverviewInfo(df):
    df['has_overview'] = df['overview'].apply(lambda x: isinstance(x, str))
    df['overview_len'] = df['overview'].apply(lambda x: len(x) if isinstance(x, str) else 0)
    return df

In [36]:
df = extractOverviewInfo(df)


In [37]:
test = extractOverviewInfo(test)

In [38]:
df['has_overview'] = df['has_overview'].astype(int)
test['has_overview'] = test['has_overview'].astype(int)
df = df.drop(['overview'], axis = 1)
test = test.drop(['overview'], axis = 1)


In [40]:
df['budget_runtime'] = (df['budget'] + 1)/(df['runtime']+1)
df['poularity_year'] = (df['popularity']+1)/(df['year']+1)
df['budget_popularity'] = (df['budget']+1)/(df['popularity']+1)
df['budget_year'] = (df['budget'] + 1)/(df['year']+1)
df['runtime_year'] = (df['runtime']+1)/(df['year']+1)

#test


In [41]:
test['budget_runtime'] = (test['budget'] + 1)/(test['runtime']+1)
test['poularity_year'] = (test['popularity']+1)/(test['year']+1)
test['budget_popularity'] = (test['budget']+1)/(test['popularity']+1)
test['budget_year'] = (test['budget'] + 1)/(test['year']+1)
test['runtime_year'] = (test['runtime']+1)/(test['year']+1)

In [45]:

def date_features(df):
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['release_month'] = df['release_date'].dt.month
    df['release_day'] = df['release_date'].dt.day
    #df['release_quarter'] = df['release_date'].dt.quarter
    df['release_dayofweek'] = df['release_date'].dt.dayofweek
    df.drop(columns=['release_date'], inplace=True)
    return df

df = date_features(df)
test = date_features(test)


In [46]:
#df['revenue'] = np.log1p(df['revenue'])
y = df[["revenue"]] #can prob get rid of this

In [47]:
members = df["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_cast"] = num_cast

In [48]:
members = df["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

df["num_crew"] = num_cast

In [49]:
num_male_cast = []
num_female_cast = []
num_unspecified_cast = []

substring_male = " 'gender': 2"
substring_female = " 'gender': 1"
substring_unspecified = " 'gender': 0"

for i in members:
    if isinstance(i, str):
        males = i.count(substring_male)
        num_male_cast.append(males)
    
        females = i.count(substring_female)
        num_female_cast.append(females)
    
        unspecified = i.count(substring_unspecified)
        num_unspecified_cast.append(unspecified)
    else:
        num_male_cast.append(0)
        num_female_cast.append(0)
        num_unspecified_cast.append(0)

df["num_male_cast"] = num_male_cast
df["num_female_cast"] = num_female_cast
df["num_unspecified_cast"] = num_unspecified_cast

df["cast_ratio_male"] = df["num_male_cast"] / (df["num_male_cast"] + df["num_female_cast"] + df["num_unspecified_cast"])
df["cast_ratio_female"] = df["num_female_cast"] / (df["num_male_cast"] + df["num_female_cast"] + df["num_unspecified_cast"])
df["cast_ratio_unspecified"] = df["num_unspecified_cast"] / (df["num_male_cast"] + df["num_female_cast"] + df["num_unspecified_cast"])

In [50]:
crew = df["crew"]

num_male_crew = []
num_female_crew = []
num_unspecified_crew = []

substring_male = "'gender\': 2"
substring_female = "'gender\': 1"
substring_unspecified = "'gender\': 0"

for i in crew:
    if isinstance(i, list):
        string = str(i)
        males = string.count(substring_male)
        num_male_crew.append(males)
    
        females = string.count(substring_female)
        num_female_crew.append(females)
    
        unspecified = string.count(substring_unspecified)
        num_unspecified_crew.append(unspecified)
    else:
        num_male_crew.append(0)
        num_female_crew.append(0)
        num_unspecified_crew.append(0)

df["num_male_crew"] = num_male_crew
df["num_female_crew"] = num_female_crew
df["num_unspecified_crew"] = num_unspecified_crew

df["crew_ratio_male"] = df["num_male_crew"] / (df["num_male_crew"] + df["num_female_crew"] + df["num_unspecified_crew"])
df["crew_ratio_female"] = df["num_female_crew"] / (df["num_male_crew"] + df["num_female_crew"] + df["num_unspecified_crew"])
df["crew_ratio_unspecified"] = df["num_unspecified_crew"] / (df["num_male_crew"] + df["num_female_crew"] + df["num_unspecified_crew"])

#move below here 


In [51]:
members = test["cast"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_cast"] = num_cast

In [52]:
members = test["crew"]

num_cast = []

for i in members:
    if isinstance(i, str):
        number = sum(1 for c in i if c == "{")
        num_cast.append(number)
    else:
        num_cast.append(0)

test["num_crew"] = num_cast

In [53]:
num_male_cast = []
num_female_cast = []
num_unspecified_cast = []

substring_male = " 'gender': 2"
substring_female = " 'gender': 1"
substring_unspecified = " 'gender': 0"

for i in members:
    if isinstance(i, list):
        males = i.count(substring_male)
        num_male_cast.append(males)
    
        females = i.count(substring_female)
        num_female_cast.append(females)
    
        unspecified = i.count(substring_unspecified)
        num_unspecified_cast.append(unspecified)
    else:
        num_male_cast.append(0)
        num_female_cast.append(0)
        num_unspecified_cast.append(0)

test["num_male_cast"] = num_male_cast
test["num_female_cast"] = num_female_cast
test["num_unspecified_cast"] = num_unspecified_cast

test["cast_ratio_male"] = test["num_male_cast"] / (test["num_male_cast"] + test["num_female_cast"] + test["num_unspecified_cast"])
test["cast_ratio_female"] = test["num_female_cast"] / (test["num_male_cast"] + test["num_female_cast"] + test["num_unspecified_cast"])
test["cast_ratio_unspecified"] = test["num_unspecified_cast"] / (test["num_male_cast"] + test["num_female_cast"] + test["num_unspecified_cast"])

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   popularity                 3000 non-null   float64
 7   production_companies       3000 non-null   object 
 8   production_countries       3000 non-null   object 
 9   runtime                    3000 non-null   float64
 10  spoken_languages           3000 non-null   object 
 11  status                     3000 non-null   int32  
 12  title                      3000 non-null   object 
 13  cast                       3000 non-null   objec

In [55]:
crew = test["crew"]

num_male_crew = []
num_female_crew = []
num_unspecified_crew = []

substring_male = "'gender\': 2"
substring_female = "'gender\': 1"
substring_unspecified = "'gender\': 0"

for i in crew:
    if isinstance(i, list):
        string = str(i)
        males = string.count(substring_male)
        num_male_crew.append(males)
    
        females = string.count(substring_female)
        num_female_crew.append(females)
    
        unspecified = string.count(substring_unspecified)
        num_unspecified_crew.append(unspecified)
    else:
        num_male_crew.append(0)
        num_female_crew.append(0)
        num_unspecified_crew.append(0)

test["num_male_crew"] = num_male_crew
test["num_female_crew"] = num_female_crew
test["num_unspecified_crew"] = num_unspecified_crew

test["crew_ratio_male"] = test["num_male_crew"] / (test["num_male_crew"] + test["num_female_crew"] + test["num_unspecified_crew"])
test["crew_ratio_female"] = test["num_female_crew"] / (test["num_male_crew"] + test["num_female_crew"] + test["num_unspecified_crew"])
test["crew_ratio_unspecified"] = test["num_unspecified_crew"] / (test["num_male_crew"] + test["num_female_crew"] + test["num_unspecified_crew"])

In [60]:
# Genre 

genres = df["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
df["genres_list"] = genres_list


In [61]:

genresInMovie = df["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
df["isAction"] = action
df["isAdventure"] = adventure
df["isAnimation"] = animation
df["isComedy"] = comedy
df["isCrime"] = crime
df["isDocumentary"] = documentary
df["isDrama"] = drama
df["isFamily"] = family
df["isFantasy"] = fantasy
df["isForeign"] = foreign
df["isHorror"] = horror
df["isMusic"] = music
df["isMystery"] = mystery
df["isRomance"] = romance
df["isSciFi"] = scifi
df["isThriller"] = thriller


In [62]:
genres = test["genres"]

first = []
second = []
third = []
fourth = []
fifth = []
sixth = []
seventh = []

for string in genres:
    if isinstance(string, str):
        splits = string.split("'")
        genre1 = splits[5]
        first.append(genre1)
        multiples1 = splits[7:]
        if not multiples1:
            second.append('NA')
        else:
            genre2 = multiples1[4]
            second.append(genre2)
        multiples2 = multiples1[7:]
        if not multiples2:
            third.append('NA')
        else:
            genre3 = multiples2[3]
            third.append(genre3)
        multiples3 = multiples2[6:]
        if not multiples3:
            fourth.append('NA')
        else:
            genre4 = multiples3[3]
            fourth.append(genre4)
        multiples4 = multiples3[5:]
        if not multiples4:
            fifth.append('NA')
        else:
            genre5 = multiples4[4]
            fifth.append(genre5)
        multiples5 = multiples4[6:]
        if not multiples5:
            sixth.append('NA')
        else:
            genre6 = multiples5[4]
            sixth.append(genre6)
        multiples6 = multiples5[6:]
        if not multiples6:
            seventh.append('NA')
        else:
            genre7 = multiples6[4]
            seventh.append(genre7)
    else:
        first.append('NA')
        second.append('NA')
        third.append('NA')
        fourth.append('NA')
        fifth.append('NA')
        sixth.append('NA')
        seventh.append('NA')

genres_list = list(zip(first, second, third, fourth, fifth, sixth, seventh))
test["genres_list"] = genres_list


In [63]:
genresInMovie = test["genres_list"]

action = []
adventure = []
animation = []
comedy = []
crime = []
documentary = []
drama = []
family = []
fantasy = []
foreign = []
horror = []
music = []
mystery = []
romance = []
scifi = []
thriller = []

for movie in genresInMovie:
    movie = list(movie)
    
    isAction = 0
    isAdventure = 0
    isAnimation = 0
    isComedy = 0
    isCrime = 0
    isDocumentary = 0
    isDrama = 0
    isFamily = 0
    isFantasy = 0
    isForeign = 0
    isHorror = 0
    isMusic = 0
    isMystery = 0
    isRomance = 0
    isSciFi = 0
    isThriller = 0
    
    for genre in movie:
        if genre == "Action":
            isAction = 1
        if genre == "Adventure":
            isAdventure = 1
        if genre == "Animation":
            isAnimation = 1
        if genre == "Comedy":
            isComedy = 1
        if genre == "Crime":
            isCrime = 1
        if genre == "Documentary":
            isDocumentary = 1
        if genre == "Drama":
            isDrama = 1
        if genre == "Family":
            isFamily = 1
        if genre == "Fantasy":
            isFantasy = 1
        if genre == "Foreign":
            isForeign = 1
        if genre == "Horror":
            isHorror = 1
        if genre == "Music":
            isMusic = 1
        if genre == "Mystery":
            isMystery = 1
        if genre == "Romance":
            isRomance = 1
        if genre == "Science Fiction":
            isSciFi = 1
        if genre == "Thriller":
            isThriller = 1
            
    action.append(isAction)
    adventure.append(isAdventure)
    animation.append(isAnimation)
    comedy.append(isComedy)
    crime.append(isCrime)
    documentary.append(isDocumentary)
    drama.append(isDrama)
    family.append(isFamily)
    fantasy.append(isFantasy)
    foreign.append(isForeign)
    horror.append(isHorror)
    music.append(isMusic)
    mystery.append(isMystery)
    romance.append(isRomance)
    scifi.append(isSciFi)
    thriller.append(isThriller)
    
test["isAction"] = action
test["isAdventure"] = adventure
test["isAnimation"] = animation
test["isComedy"] = comedy
test["isCrime"] = crime
test["isDocumentary"] = documentary
test["isDrama"] = drama
test["isFamily"] = family
test["isFantasy"] = fantasy
test["isForeign"] = foreign
test["isHorror"] = horror
test["isMusic"] = music
test["isMystery"] = mystery
test["isRomance"] = romance
test["isSciFi"] = scifi
test["isThriller"] = thriller

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 61 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3000 non-null   int64  
 1   budget                     3000 non-null   int64  
 2   genres                     3000 non-null   object 
 3   imdb_id                    3000 non-null   object 
 4   original_language          3000 non-null   int32  
 5   original_title             3000 non-null   object 
 6   popularity                 3000 non-null   float64
 7   production_companies       3000 non-null   object 
 8   production_countries       3000 non-null   object 
 9   runtime                    3000 non-null   float64
 10  spoken_languages           3000 non-null   object 
 11  status                     3000 non-null   int32  
 12  title                      3000 non-null   object 
 13  cast                       3000 non-null   objec

In [ ]:
'''
Tried these features not much difference 
df['log_popularity'] = np.log1p(df['popularity'])
test['log_popularity'] = np.log1p(test['popularity'])
df['log_budget'] = np.log1p(df['budget'])

df['ratio_budget_runtime'] = (df['log_budget'] / df['runtime'])
df['ratio_budget_popularity'] = df['log_budget'] / df['log_popularity']
df['ratio_budget_year'] = df['log_budget'] / df['release_year'] #
df['budget_to_mean_year'] = df['log_budget'] / df.groupby('release_year')['log_budget'].transform('mean')
df['ratio_popularity_year'] = df['log_popularity'] / df['release_year']
df['popularity_to_mean_year'] = df['log_popularity'] / df.groupby('release_year')['log_popularity'].transform('mean')

df['ratio_budget_year'].fillna(df['ratio_budget_year'].mean(),inplace=True)
df['budget_to_mean_year'].fillna(df['budget_to_mean_year'].mean(),inplace=True) #fill na with test vals?
df['ratio_budget_runtime'].fillna(df['ratio_budget_runtime'].mean(),inplace=True) #fill na with test vals?

le = LabelEncoder()
df['prodcomp']=le.fit_transform(df['production_companies'])
test['prodcomp']=le.fit_transform(test['production_companies'])

gender of cast feature might help
'''

In [ ]:
df.head()

In [ ]:
df.info()

In [142]:
X = df[["popularity", "runtime", "budget", "isbelongs_to_collectionNA", "isTaglineNA","budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller", 'release_month', 'release_day','release_dayofweek'  
        , "status", 'original_language',"crew_ratio_male", 'num_female_cast', 'num_male_cast', 'num_female_crew', 'num_male_crew']]
# "isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller"
#X random forest model for some reason cannot handle the num_female_cast, num_male_cast etc variables, you will get an error if
# you have those features in the X

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#xgbr.fit(X_train, y_train)

In [140]:
m = RandomForestRegressor(n_estimators=60, random_state = 42, min_samples_leaf = 5,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

C:\Users\Ben\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(min_samples_leaf=5, n_estimators=60, n_jobs=-1,
                      oob_score=True, random_state=42)

In [71]:
def rmse(y_pred, y_true):
    return np.sqrt(mean_squared_error(y_pred, y_true))

def print_rf_score(model):
    print(f'Train R2:   {model.score(X_train, y_train)}')
    print(f'Test R2:   {model.score(X_test, y_test)}')
    print(f'Train RMSE: {rmse(model.predict(X_train), y_train)}')
    print(f'Test RMSE: {rmse(model.predict(X_test), y_test)}')

In [141]:
print_rf_score(m)

Train R2:   0.7710002981847681
Test R2:   0.5422207268164349
Train RMSE: 1.4729909716043725
Test RMSE: 2.0198803099460236


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [50,60,70,80,90,100,110,120,130,140,150,200,400, 500],
    'max_depth' : [5,6,7,8,9,10,11,15,20],
    'min_samples_leaf' : [4,5,6,7,8,9,10,15]
}
CV_rfc = GridSearchCV(estimator=m, param_grid=param_grid, cv= 5, verbose = 2, scoring = 'neg_mean_squared_error')
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_estimator_)
bestResults = pd.DataFrame(CV_rfc.cv_results_)


In [ ]:
#best RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=500,
                     # n_jobs=-1, oob_score=True, random_state=42)

In [ ]:
m2 = RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=500,
                      n_jobs=-1, oob_score=True, random_state=42)
m2.fit(X_train, y_train)

In [ ]:
print_rf_score(m2)

In [127]:
xgb3 = XGBRegressor(
        n_estimators = 10000, 
        learning_rate = 0.001,
        max_depth = 6, 
        subsample = 0.3, #.3
        colsample_bytree = 0.6,
        eta = .1,
        )
xgb3.fit(X_train, y_train)

[11:29:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [128]:
print_rf_score(xgb3)

Train R2:   0.6669720531158481
Test R2:   0.5223153652874217
Train RMSE: 1.7763272020769652
Test RMSE: 2.06332770181094


In [89]:
xgbexp = XGBRegressor(objective = 'reg:linear',
                                  eta = 0.01,
                                  max_depth = 6,
                                  min_child_weight = 3,
                                  subsample = 0.8,
                                  colsample_bytree = 0.7,
                                  eval_metric = 'rmse',
                                  n_estimators = 65)
xgbexp.fit(X_train, y_train)

[11:21:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.7, eta=0.01, eval_metric='rmse', max_depth=6,
             min_child_weight=3, n_estimators=65, subsample=0.8)

In [90]:
print_rf_score(xgbexp)

Train R2:   0.84042120788066
Test R2:   0.5388474805448226
Train RMSE: 1.2296180034534547
Test RMSE: 2.027308614773055


In [ ]:
'''
try this 

(objective = 'reg:linear',
                                  eta = 0.01,
                                  max_depth = 6,
                                  min_child_weight = 3,
                                  subsample = 0.8,
                                  colsample_bytree = 0.7,
                                  eval_metric = 'rmse',
                                  seed = 1,
                                  n_estimators = 2800)
'''

In [ ]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [ ]:
model = XGBRegressor()
param_grid = {
        'n_estimators': [10000],
        'min_child_weight': [1, 5, 10],
        #"subsample" : [.3, .6, 1.0],
        'gamma': [0.5, 1, 1.5, 2],
        "eta": [0.01, 0.1, 1],
        #'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5,6,7]
}

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

In [ ]:
# results 'eta': 0.01, 'gamma': 2, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 10000

In [101]:
xgb_full_train = XGBRegressor(objective = 'reg:squarederror',
                                  eta = 0.01,
                                  max_depth = 6,
                                  min_child_weight = 3,
                                  subsample = 0.8,
                                  colsample_bytree = 0.7,
                                  eval_metric = 'rmse',
                                  seed = 1,
                                  n_estimators = 2800)

In [102]:
xgb_full_train.fit(X_train, y_train)

XGBRegressor(colsample_bytree=0.7, eta=0.01, eval_metric='rmse', max_depth=6,
             min_child_weight=3, n_estimators=2800,
             objective='reg:squarederror', seed=1, subsample=0.8)

In [103]:
print_rf_score(xgb_full_train)

Train R2:   0.9999999631037635
Test R2:   0.5206689593013087
Train RMSE: 0.0005912534133853876
Test RMSE: 2.066880414986628


In [ ]:
\

In [ ]:
print_rf_score(xgb)

In [105]:
predictdf= test[["popularity", "runtime", "budget", "isbelongs_to_collectionNA", "isTaglineNA","budget_year_ratio", "year",'budget_runtime', 'poularity_year', 'budget_popularity','budget_year','runtime_year', 'has_overview', 'overview_len', "num_cast", 'num_crew',"isAction", "isComedy", "isCrime", "isDocumentary", 'isDrama', "isFamily","isFantasy", "isForeign", "isHorror", "isMusic", "isMystery","isRomance", "isSciFi", "isThriller", 'release_month', 'release_day','release_dayofweek'  
        , "status", 'original_language', "crew_ratio_male", 'num_female_cast', 'num_male_cast', 'num_female_crew', 'num_male_crew']]

In [97]:
# results 'eta': 0.01, 'gamma': 2, 'max_depth': 7, 'min_child_weight': 10, 'n_estimators': 10000
xgb2 = XGBRegressor(
        min_child_weight = 10,
        max_depth = 7, 
        subsample = 0.3, #.3
        colsample_bytree = 0.8,
        gamma = 2,
        n_estimators = 5000,
        eta = 0.01)

In [99]:
xgb2.fit(X,y)

[11:23:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.8, eta=0.01, gamma=2, max_depth=7,
             min_child_weight=10, n_estimators=5000, subsample=0.3)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators = 20, max_depth = 8, min_samples_leaf=5)
reg.fit(X_train, y_train)

In [ ]:
print_rf_score(reg)

In [ ]:
xgb2.fit(X_train,y_train)

In [100]:
print_rf_score(xgb2)

Train R2:   0.9845222617661307
Test R2:   0.9841709331063097
Train RMSE: 0.3829449751142106
Test RMSE: 0.3755996943853204


In [ ]:
predictdf

In [ ]:
predictarray = np.asarray(predictdf)

In [ ]:

submission = m2.predict(predictdf)
submission


In [ ]:
submissionreg = reg.predict(predictdf)

In [129]:
xgbsubmission = xgb3.predict(predictdf)
xgbsubmission

array([13.361865, 12.876231, 13.779128, ..., 16.09991 , 13.097938,
       12.501298], dtype=float32)

In [130]:
subdf = pd.DataFrame(xgbsubmission)
subdf

,0
0,13.361865
1,12.876231
2,13.779128
3,13.415666
4,12.808491
...,...
4393,15.738279
4394,15.608645
4395,16.099911
4396,13.097938


In [131]:
subdf["id"] = test["id"]
subdf.columns = ['revenue', 'id']

In [132]:
subdf = subdf[['id', 'revenue']]

In [133]:
subdf['revenue'] = np.round(np.expm1(subdf["revenue"]))

In [134]:
subdf

,id,revenue
0,3001,635307.0
1,3002,390908.0
2,3003,964270.0
3,3004,670423.0
4,3005,365305.0
...,...,...
4393,7394,6839870.0
4394,7395,6008257.0
4395,7396,9819793.0
4396,7397,487934.0


In [135]:
subdf.to_csv("Submission14.csv")

In [ ]:
# month released 
# week released etc. 
# number of keywords 
# cast gender? 